<img width="8%" alt="Naas.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Naas.png" style="border-radius: 15%">

# Plugin

**Tags:** #naaschatplugin #naas #naas_driver #chat #plugin #ai

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Description:** This notebook creates a Naas Chat plugin with commands.

## Input

### Import libraries

In [ ]:
import naas
from naas_drivers import naas_chat_plugin, gsheet
import naas_data_product
from IPython.display import Markdown
from datetime import date, timedelta
import os
import pandas as pd

### Setup variables
**Inputs**
- `input_dir`: Input directory to retrieve file from.
- `file_content`: Name of the file to be retrieved.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name`: Google Sheets sheet name.
- `name`: The name of the plugin.
- `system_prompt`: Plugin system prompt with the commands they can use.
- `model`: This variable holds the openai model.
- `temperature`: This variable holds the temperate value.
- `description`: This variable holds a string that describes the plugin.
- `avatar`: This variable holds a URL to an image to be displayed in your plugin as avatar.
- `model_dir`: This variable holds a relative path to the directory where input notebooks with commands will be stored.

**Outputs**
- `output_dir`: This variable holds a relative path to the directory where output files will be stored. 
- `output_path`: This variable holds the full path of the plugin.

In [ ]:
# Inputs
input_dir = os.path.join(naas_data_product.OUTPUTS_PATH, "content-engine", date.today().isoformat())
file_content = "content"
spreadsheet_url = naas.secret.get("ABI_SPREADSHEET") or "YOUR_GOOGLE_SPREADSHEET_URL"
sheet_name = "CONTENT"
name = "📲 Content Assistant"
model = "gpt-4"
temperature = 0.5
description = "Track your content performance across platforms, leverage AI for optimized strategies and assistance in writing impactful new content that will increase reach & boost engagement."
avatar = ""
model_dir = os.path.join(naas_data_product.ROOT_PATH, "models", "content-engine")

# Outputs
output_dir = os.path.join(naas_data_product.ROOT_PATH, "outputs", "plugins")
os.makedirs(output_dir, exist_ok=True) # Create dirs
output_path = os.path.join(output_dir, f"{name.lower().replace(' ', '_')}.json")

## Model

### Get image

In [ ]:
image_url = get_image_asset(input_dir, "content_views_trend.png")
print("Image URL:", image_url)

image_html = get_image_asset(input_dir, "content_views_trend.html")
print("Image HTML:", image_html)

### Get data

In [ ]:
def get_data(
    input_dir,
    input_file,
    spreadsheet_url,
    sheet_name
):
    # Init
    df = pd.DataFrame()
    
    # Load data from pickle
    data = pload(input_dir, input_file)    
    if data is None:
        data = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name)
    if isinstance(data, pd.DataFrame):
        df = data.copy()

    # Filter on this week and last week
    if len(df) > 0:
        tw = date.today().strftime("W%W-%Y")
        lw = (date.today() - timedelta(days=date.today().weekday() + 7)).strftime("W%W-%Y")
        df = df[df["SCENARIO"].isin([tw, lw])]
    return df.reset_index(drop=True)

data = get_data(input_dir, file_content, spreadsheet_url, sheet_name)
print("Data:", len(data))
data.head(len(data))

### Create prompt

In [ ]:
system_prompt = f"""
Act as a Content Assistant who has access to valuable data and insights about the content strategy. 
Your role is to manage and optimize the content, ensuring it reaches the target audience effectively. 
You will need to analyze user behavior and preferences to understand their needs and interests. 
Utilize the data from various sources to personalize the content strategy and track its performance. 
Your ultimate goal is to create engaging and relevant content that resonates with the audience, contributing to the overall success of the content strategy. 
The first message should be about presenting yourself with a maximum of 3 bullet points and displaying the current content analytics data to be displayed as an image inside the markdown of the chat: ![{image_url}]({image_html}).
You must ALWAYS show the image in the first message.
Then, wait for the first answer from the user, and then start with the first high-level analysis. 
Here is the data from the content analytics that you should focus on: {data.to_string()}
"""
system_prompt

### Create commands

In [ ]:
commands = create_command(model_dir)

## Output

### Create Naas Chat plugin
This function will generate the plugin in JSON format and also verify if your prompt adheres to the recommended limit, which is set at 20% of the maximum tokens allowed by the model. Then, it will save your plugin in your local environment.

In [ ]:
# Create plugin
plugin_file_path = naas_chat_plugin.create_plugin(
    name=name,
    prompt=system_prompt,
    model=model,
    temperature=temperature,
    output_path=output_path,
    description=description,
    avatar=avatar,
    commands=commands
)

### Create asset
This asset can be utilized by using the command `/use` in your Naas Chat or by simply clicking on the link provided in the last cell output.

In [ ]:
plugin_url = naas.asset.add(plugin_file_path, params={"inline": True})
print(plugin_url)

### Create new chat
NB: You don't need to click on 'Create New Chat' everytime you update your system prompt, you can use the command `/refresh`.

In [ ]:
Markdown(f"[Create New Chat](https://naas.ai/chat/use?plugin_url={plugin_url})")